In [ ]:
image_paths = [
    r"..\treno_sdss_image_120.17063548933_0.358096706241307.png",
    r"..\data\skyserver\dr18\120_0.2\sdss_image_180.171408773627_0.35639930317447.jpg",
    r"..\data\skyserver\dr18\200_0.2\sdss_image_180.171408773627_0.35639930317447.jpg",
    r"..\data\skyserver\dr18\120_0.2\sdss_image_180.314025750627_-0.0257549129402702.jpg",
    r"..\data\skyserver\dr18\120_0.4\sdss_image_180.313186780449_0.206615990437186.jpg",
    r"..\data\skyserver\dr18\120_0.4\sdss_image_180.316130449752_-0.0240209645983516.jpg",
    r"..\data\skyserver\dr18\120_0.3\sdss_image_180.731788727317_0.268910644954916.jpg",
    r"..\data\skyserver\dr18\120_0.2\sdss_image_180.316210953946_0.234630238900774.jpg",
    r"..\data\skyserver\dr18\120_0.2\sdss_image_180.731788727317_0.268910644954916.jpg",
    r"..\data\skyserver\dr18\120_0.05\sdss_image_180.866545948245_0.0733494204912493.jpg",
    r"..\data\skyserver\dr18\120_0.2\sdss_image_180.866302981297_-0.139722107311907.jpg",
    r"..\data\skyserver\compromised\3-types-of-blurring\blurred_average_274-51913-230.jpg",
    r"..\data\skyserver\dr18\120_0.3\sdss_image_180.866514381438_0.345166549524844.jpg",
    r"..\data\skyserver\dr18\120_0.2\sdss_image_180.176482873675_0.182726662479744.jpg",
    r"..\data\dfot\Cutout Files\star\grb0422a_01_68_1020_6.jpg",
    r"..\data\dfot\Cutout Files\star\grb0422a_01_546_218_6.jpg",
    r"..\data\dfot\Cutout Files\star\IC745_H01_1526_1132_6.jpg",
    r"..\data\skyserver\dr18\120_0.3\sdss_image_180.176390275064_-0.047091687826631.jpg",
    r"..\data\skyserver\dr18\120_0.4\sdss_image_180.867871194172_0.341396873315708.jpg",
    r"..\data\skyserver\dr18\120_0.4\sdss_image_180.867050000237_0.314017431674984.jpg",
    r"..\data\skyserver\dr18\120_0.4\sdss_image_180.451206348309_-0.324013595202261.jpg",
    r"..\data\skyserver\clean_200_0.2\sdss_image_49.2511248902923_43.8591858289443.jpg"
]

In [16]:
import cv2
import numpy as np

def resize_and_add_border(image, target_height, target_width, border_size=10):
    """
    Redimensiona a imagem para o tamanho exato especificado, adicionando uma borda branca ao redor.
    
    Parâmetros:
    - image: imagem original a ser redimensionada e com borda adicionada.
    - target_height: altura alvo da imagem redimensionada sem borda.
    - target_width: largura alvo da imagem redimensionada sem borda.
    - border_size: tamanho da borda branca a ser adicionada.
    
    Retorna:
    - Imagem redimensionada com borda branca.
    """
    h, w = image.shape[:2]
    scaling_factor = min((target_width - 2 * border_size) / w, (target_height - 2 * border_size) / h)
    new_size = (int(w * scaling_factor), int(h * scaling_factor))
    resized_image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    
    # Cria uma imagem branca com o tamanho exato do alvo + borda
    result = np.ones((target_height + 2 * border_size, target_width + 2 * border_size, 3), dtype=np.uint8) * 255
    y_offset = (target_height + 2 * border_size - new_size[1]) // 2
    x_offset = (target_width + 2 * border_size - new_size[0]) // 2
    result[y_offset:y_offset + new_size[1], x_offset:x_offset + new_size[0]] = resized_image
    return result

def create_image_mosaic(image_list, rows, cols, target_height=120, target_width=120, border_size=10):
    """
    Cria um mosaico de imagens com a lista de imagens fornecida, redimensionando-as e adicionando uma borda branca.
    
    Parâmetros:
    - image_list: lista de imagens (cada imagem deve ter o mesmo tamanho após redimensionamento e adição de borda).
    - rows: número de linhas no mosaico.
    - cols: número de colunas no mosaico.
    - target_height, target_width: dimensões alvo para redimensionar cada imagem (sem a borda).
    - border_size: tamanho da borda branca em torno de cada imagem.
    
    Retorna:
    - Uma imagem única que representa o mosaico.
    """
    
    # Carrega, redimensiona e adiciona borda às imagens para o tamanho alvo
    image_list = [resize_and_add_border(cv2.imread(image_path), target_height, target_width, border_size) for image_path in image_list]
    if any(img is None for img in image_list):
        raise ValueError("Uma ou mais imagens não foram carregadas corretamente. Verifique os caminhos dos arquivos.")
    
    # Calcula as novas dimensões com borda incluída
    img_height, img_width = target_height + 2 * border_size, target_width + 2 * border_size
    
    # Cria uma imagem branca para o mosaico com base no tamanho das imagens e na disposição
    mosaic = np.ones((img_height * rows, img_width * cols, 3), dtype=np.uint8) * 255  # Fundo branco
    
    # Adiciona cada imagem ao mosaico
    for i in range(rows):
        for j in range(cols):
            # Calcula o índice da imagem na lista
            index = i * cols + j
            if index < len(image_list):
                # Define a posição onde a imagem será colocada no mosaico
                y = i * img_height
                x = j * img_width
                mosaic[y:y + img_height, x:x + img_width] = image_list[index]
    
    return mosaic

# Carrega as imagens e cria o mosaico
mosaic_image = create_image_mosaic(image_paths, rows=5, cols=5, target_height=120, target_width=120, border_size=2)

# Salva o mosaico em um arquivo
output_path = r"resultado_mosaico.jpg"
cv2.imwrite(output_path, mosaic_image)
print(f"Mosaico salvo em {output_path}")


Mosaico salvo em resultado_mosaico.jpg


In [ ]:
import cv2
import numpy as np
import os

def resize_and_add_border(image, target_height, target_width, border_size=10):
    """
    Redimensiona a imagem para o tamanho exato especificado, adicionando uma borda branca ao redor.
    
    Parâmetros:
    - image: imagem original a ser redimensionada e com borda adicionada.
    - target_height: altura alvo da imagem redimensionada sem borda.
    - target_width: largura alvo da imagem redimensionada sem borda.
    - border_size: tamanho da borda branca a ser adicionada.
    
    Retorna:
    - Imagem redimensionada com borda branca.
    """
    h, w = image.shape[:2]
    scaling_factor = min((target_width - 2 * border_size) / w, (target_height - 2 * border_size) / h)
    new_size = (int(w * scaling_factor), int(h * scaling_factor))
    resized_image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    
    # Cria uma imagem branca com o tamanho exato do alvo + borda
    result = np.ones((target_height + 2 * border_size, target_width + 2 * border_size, 3), dtype=np.uint8) * 255
    y_offset = (target_height + 2 * border_size - new_size[1]) // 2
    x_offset = (target_width + 2 * border_size - new_size[0]) // 2
    result[y_offset:y_offset + new_size[1], x_offset:x_offset + new_size[0]] = resized_image
    return result

def create_image_mosaic_from_directory(directory_path, rows, cols, target_height=120, target_width=120, border_size=10):
    """
    Cria um mosaico de imagens a partir de imagens em um diretório, redimensionando-as e adicionando uma borda branca.
    
    Parâmetros:
    - directory_path: caminho do diretório contendo as imagens.
    - rows: número de linhas no mosaico.
    - cols: número de colunas no mosaico.
    - target_height, target_width: dimensões alvo para redimensionar cada imagem (sem a borda).
    - border_size: tamanho da borda branca em torno de cada imagem.
    
    Retorna:
    - Uma imagem única que representa o mosaico.
    """
    
    # Lista todas as imagens no diretório
    image_paths = [os.path.join(directory_path, fname) for fname in os.listdir(directory_path) 
                   if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    # Carrega, redimensiona e adiciona borda às imagens para o tamanho alvo
    image_list = [resize_and_add_border(cv2.imread(image_path), target_height, target_width, border_size) 
                  for image_path in image_paths]
    if any(img is None for img in image_list):
        raise ValueError("Uma ou mais imagens não foram carregadas corretamente. Verifique os caminhos dos arquivos.")
    
    # Calcula as novas dimensões com borda incluída
    img_height, img_width = target_height + 2 * border_size, target_width + 2 * border_size
    
    # Cria uma imagem branca para o mosaico com base no tamanho das imagens e na disposição
    mosaic = np.ones((img_height * rows, img_width * cols, 3), dtype=np.uint8) * 255  # Fundo branco
    
    # Adiciona cada imagem ao mosaico
    for i in range(rows):
        for j in range(cols):
            # Calcula o índice da imagem na lista
            index = i * cols + j
            if index < len(image_list):
                # Define a posição onde a imagem será colocada no mosaico
                y = i * img_height
                x = j * img_width
                mosaic[y:y + img_height, x:x + img_width] = image_list[index]
    
    return mosaic

# Define o caminho do diretório de imagens e cria o mosaico
directory_path = r"kernel_param\similaridade\stars_with_similar_intensity\mosaico"
mosaic_image = create_image_mosaic_from_directory(directory_path, rows=5, cols=5, target_height=120, target_width=120, border_size=2)

# Salva o mosaico em um arquivo
output_path = r"Resultado_mosaico_ImagensItensidadeSimilar.jpg"
cv2.imwrite(output_path, mosaic_image)
print(f"Mosaico salvo em {output_path}")


Mosaico salvo em Resultado_mosaico_ImagensItensidadeSimilar.jpg
